# 📊 Análisis de Opciones – IOL (Pandas)

Pipeline completo para:
- obtener opciones desde IOL
- filtrar iliquidez
- normalizar strikes
- calcular moneyness correctamente
- clasificar ATM / ITM / OTM
- rankear oportunidades reales


In [1]:
import re
import pandas as pd
import numpy as np
from src.iol.container import iol_client


## 1️⃣ Fetch de datos

In [2]:
options = await iol_client.fetch_all_options()
len(options)


1506

## 2️⃣ Normalización inicial

In [3]:
rows = []

for op in options:
    rows.append({
        "symbol": op.symbol,
        "last_price": op.last_price,
        "variation": op.variation,
        "volume": op.volume,
        "trade_count": op.trade_count,
        "timestamp": op.timestamp,
    })

df = pd.DataFrame(rows)
df.head()


,symbol,last_price,variation,volume,trade_count,timestamp
0,ALUC1000AB,0.0,0.00,0.0,0.0,2025-10-24 08:00:00.000
1,ALUC1000EN,0.0,0.00,0.0,0.0,2025-11-26 08:00:00.000
2,ALUC1000FE,95.0,-26.95,0.0,6.0,2025-12-23 16:45:42.380
3,ALUC1000JU,0.0,0.00,0.0,0.0,2025-12-22 08:00:00.000
4,ALUC1050AB,0.0,0.00,0.0,0.0,2025-10-24 08:00:00.000


## 3️⃣ Filtro duro de viabilidad

In [4]:
df = df[
    (df["last_price"] > 0) &
    (df["volume"] >= 5) &
    (df["trade_count"] >= 2)
].copy()

len(df)


40

## 4️⃣ Parseo vectorizado del símbolo

In [5]:
symbol_re = re.compile(r"(?P<underlying>[A-Z]+)(?P<cp>[CV])(?P<strike>\d+)")

parsed = df["symbol"].str.extract(symbol_re)

df["underlying"] = parsed["underlying"]
df["type"] = parsed["cp"].map({"C": "CALL", "V": "PUT"})
df["raw_strike"] = parsed["strike"].astype(float)

df = df.dropna(subset=["underlying", "type", "raw_strike"])


## 5️⃣ Precio del subyacente

In [6]:
spot_prices = {
    "GFG": 8325.0,
    "YPF": 54900.0,
}

df["spot"] = df["underlying"].map(spot_prices)
df = df.dropna(subset=["spot"])


## 6️⃣ Normalización del strike

In [7]:
df["strike"] = np.where(
    df["raw_strike"] > df["spot"] * 3,
    df["raw_strike"] / 10,
    df["raw_strike"],
)


## 7️⃣ Cálculo correcto de moneyness

In [8]:
df["moneyness"] = np.where(
    df["type"] == "CALL",
    (df["spot"] - df["strike"]) / df["spot"],
    (df["strike"] - df["spot"]) / df["spot"],
)

assert df["moneyness"].between(-1, 1).all()


## 8️⃣ Clasificación ATM / ITM / OTM

In [9]:
df["bucket"] = np.select(
    [
        df["moneyness"].abs() < 0.03,
        df["moneyness"] > 0,
    ],
    ["ATM", "ITM"],
    default="OTM",
)


## 9️⃣ Scoring

In [10]:
df["score"] = (
    df["variation"].clip(upper=50) +
    (df["volume"] * 2).clip(upper=40) +
    np.where(df["bucket"] == "ATM", 20, 0) +
    np.where(df["bucket"] == "OTM", 10, 0)
)


## 🔟 Resultado final

In [11]:
cols = [
    "symbol",
    "underlying",
    "type",
    "strike",
    "spot",
    "moneyness",
    "bucket",
    "last_price",
    "variation",
    "volume",
    "trade_count",
    "score",
]

df_final = df.sort_values("score", ascending=False)[cols]
df_final.head(10)


,symbol,underlying,type,strike,spot,moneyness,bucket,last_price,variation,volume,trade_count,score
731,GFGC85539F,GFG,CALL,8553.9,8325.0,-0.027495,ATM,560.000,1.13,1580.0,243.0,61.13
726,GFGC82539F,GFG,CALL,8253.9,8325.0,0.008541,ATM,709.780,0.17,193.0,52.0,60.17
817,GFGV85539F,GFG,PUT,8553.9,8325.0,0.027495,ATM,570.000,-1.91,38.0,20.0,58.09
811,GFGV82539F,GFG,PUT,8253.9,8325.0,-0.008541,ATM,420.000,-3.23,186.0,33.0,56.77
769,GFGV53539F,GFG,PUT,5353.9,8325.0,-0.356889,OTM,3.199,5.85,143.0,72.0,55.85
712,GFGC75539F,GFG,CALL,7553.9,8325.0,0.092625,ITM,1168.423,13.08,622.0,37.0,53.08
787,GFGV69539F,GFG,PUT,6953.9,8325.0,-0.164697,OTM,63.990,-0.52,169.0,33.0,49.48
671,GFGC11777F,GFG,CALL,11777.0,8325.0,-0.414655,OTM,46.000,-0.57,4069.0,308.0,49.43
792,GFGV71767F,GFG,PUT,7176.7,8325.0,-0.137934,OTM,95.000,-0.84,340.0,78.0,49.16
806,GFGV79539F,GFG,PUT,7953.9,8325.0,-0.044577,OTM,295.500,-0.87,1501.0,191.0,49.13


## 🧠 Notas finales

- Si `moneyness` sale fuera de [-1, 1] → hay bug real
- ATM (~±3%) es la zona de mayor convexidad
- Este ranking **decide qué mirar**, no qué operar
